In [ ]:
!pip install transformers

In [ ]:
!pip install -U sentence-transformers

In [ ]:
from sentence_transformers import SentenceTransformer
sentences = ["This is an example sentence", "Each sentence is converted"]

model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
embeddings = model.encode(sentences)
print(embeddings)



In [ ]:
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=2, random_state=0).fit(embeddings)

print(kmeans.labels_)

In [ ]:
example_words = ["science", "orange", "technology", "ass", "apple", "mango", "Einstein", "math", "water", "watermelon", "chest"]


example_embeddings = model.encode(example_words)

kmeans = KMeans(n_clusters=3, random_state=0).fit(example_embeddings)

print(kmeans.labels_)

In [ ]:
import numpy as np

cosine_similarity = np.dot(example_embeddings, example_embeddings.T) / (np.linalg.norm(example_embeddings) * np.linalg.norm(example_embeddings))

cosine_similarity



In [ ]:
x_min = np.min(cosine_similarity[0, :])
x_max = np.max(cosine_similarity[0, :])

x_normalized = (cosine_similarity - x_min) / (x_max - x_min)

print(x_normalized)

In [ ]:
!pip install einops datasets

In [5]:
import torch
import torch.nn.functional as F
from transformers import AutoTokenizer, AutoModel

# Each query needs to be accompanied by an corresponding instruction describing the task.
task_name_to_instruct = {"example": "Given a question, retrieve passages that answer the question",}

query_prefix = "Instruct: "+task_name_to_instruct["example"]+"\nQuery: "
queries = [
    'are judo throws allowed in wrestling?', 
    'how to become a radiology technician in michigan?',
    'How to become better in Footbal?'
    ]

# No instruction needed for retrieval passages
passage_prefix = ""
passages = [
    "Since you're reading this, you are probably someone from a judo background or someone who is just wondering how judo techniques can be applied under wrestling rules. So without further ado, let's get to the question. Are Judo throws allowed in wrestling? Yes, judo throws are allowed in freestyle and folkstyle wrestling. You only need to be careful to follow the slam rules when executing judo throws. In wrestling, a slam is lifting and returning an opponent to the mat with unnecessary force.",
    "Below are the basic steps to becoming a radiologic technologist in Michigan:Earn a high school diploma. As with most careers in health care, a high school education is the first step to finding entry-level employment. Taking classes in math and science, such as anatomy, biology, chemistry, physiology, and physics, can help prepare students for their college studies and future careers.Earn an associate degree. Entry-level radiologic positions typically require at least an Associate of Applied Science. Before enrolling in one of these degree programs, students should make sure it has been properly accredited by the Joint Review Committee on Education in Radiologic Technology (JRCERT).Get licensed or certified in the state of Michigan.",
    "You should work on passing and dribbling a lot."
]

# load model with tokenizer
model = AutoModel.from_pretrained('nvidia/NV-Embed-v2', trust_remote_code=True)

# get the embeddings
max_length = 32768
query_embeddings = model.encode(queries, instruction=query_prefix, max_length=max_length)
passage_embeddings = model.encode(passages, instruction=passage_prefix, max_length=max_length)

# normalize embeddings
query_embeddings = F.normalize(query_embeddings, p=2, dim=1)
passage_embeddings = F.normalize(passage_embeddings, p=2, dim=1)

# get the embeddings with DataLoader (spliting the datasets into multiple mini-batches)
# batch_size=2
# query_embeddings = model._do_encode(queries, batch_size=batch_size, instruction=query_prefix, max_length=max_length, num_workers=32, return_numpy=True)
# passage_embeddings = model._do_encode(passages, batch_size=batch_size, instruction=passage_prefix, max_length=max_length, num_workers=32, return_numpy=True)

scores = (query_embeddings @ passage_embeddings.T) * 100
print(scores.tolist())
# [[87.42693328857422, 0.46283677220344543], [0.965264618396759, 86.03721618652344]]


Loading checkpoint shards: 100%|██████████| 4/4 [00:06<00:00,  1.63s/it]


[[87.42694854736328, 0.4628347158432007, 2.4941823482513428], [0.9652664661407471, 86.0372314453125, 7.357072830200195], [3.4475162029266357, 10.905562400817871, 47.55426788330078]]


In [ ]:
import math

words = ["data analysis", "data science", "machine learning", "deep learning", "artificial intelligence", "watermelon", "chest", "apple", "mango", "orange"]

embeddings1 = model.encode(words)

# normalized_embeddings = F.normalize(embeddings1, p=2, dim=1)

# scores1 = (normalized_embeddings @ normalized_embeddings.T) * 100

scores1 = F.cosine_similarity(embeddings1[9], embeddings1[8], dim=-1)

print(scores1.tolist())


/home/sercan/.cache/huggingface/modules/transformers_modules/nvidia/NV-Embed-v2/5130cf1daf847c1bacee854a6ef1ca939e747fb2/modeling_nvembed.py:349: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  'input_ids': torch.tensor(batch_dict.get('input_ids').to(batch_dict.get('input_ids')).long()),
/home/sercan/anaconda3/envs/general/lib/python3.13/contextlib.py:109: FutureWarning: `torch.backends.cuda.sdp_kernel()` is deprecated. In the future, this context manager will be removed. Please see `torch.nn.attention.sdpa_kernel()` for the new context manager, with updated signature.
  self.gen = func(*args, **kwds)


0.7909797430038452
tensor(9.0448e-05)
